## NFW profile normalised Fourier transforms

In [ ]:
# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt

# Imports from the halo model repo
sys.path.append('../src/')
import halomodel as halo

In [ ]:
# Parameters

# k range [h/Mpc]
kmin = 1e-1; kmax = 1e1
nk = 129

# Halo parameters
M_fid = 1e14

# Conversion between r and k space (approximate)
rk = 1.

# Cosmology
halo_definition = 'Mvir'
Dv = 330.
Om_m = 0.3
z = 0.

Setup values for an NFW profile:
$\rho(r)=\frac{\rho_{\rm s}}{r/r_{\rm s}(1+r/r_{\rm s})^2}$
The halo is truncated at the virial radius, `rv_fid` and $r_{\rm s}$ is defined using $r_{\rm s}= r_{\rm v}/c$, where c is the concentraion, taken from [Duffy et al. (2008)](https://arxiv.org/abs/0804.2486).

In [ ]:
# Fiducial values
hmod = halo.model(z, Om_m, Dv=Dv)

# Find the virial radius for the given halo mass: M_fid, halo definition: Dv and matter density parameter: Om_m
rv_fid = hmod.virial_radius(M_fid)
c_fid = halo.concentration(M_fid, z, halo_definition=halo_definition)
rs_fid = rv_fid/c_fid
kv_fid = rk/rv_fid
ks_fid = rk/rs_fid

# Print to screen
print('Fiducial halo')
print('Mass [log10(Msun/h)]):', np.log10(M_fid))
print('Concentration:', c_fid)
print('Virial radius [Mpc/h]:', rv_fid)
print('Scale radius [Mpc/h]:', rs_fid)
print('Virial wavenumber [h/Mpc]:', kv_fid)
print('Scale wavenumber [h/Mpc]:', ks_fid)
print()

# Fill arrays
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

# get the Fourier transform of the NFW profile
Wk = halo.window_function(ks, rv_fid, c_fid, profile='NFW')

# Varying halo mass
Mmin, Mmax = M_fid/10., M_fid*10.
nM = 11
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# Fill Wk arrays with varying M
rvs = hmod.virial_radius(Ms)
cs = halo.concentration(Ms, z, halo_definition=halo_definition)
Wks_M = halo.window_function(ks, rvs, cs, profile='NFW')

print('Minimum-mass halo')
print('Mass [log10(Msun/h)]):', np.log10(Ms[0]))
print('Concentration:', cs[0])
print('Virial radius [Mpc/h]:', rvs[0])
print()

print('Maximum-mass halo')
print('Mass [log10(Msun/h)]):', np.log10(Ms[-1]))
print('Concentration:', cs[-1])
print('Virial radius [Mpc/h]:', rvs[-1])
print()

Now lets make the figure.

In [ ]:
# Plot parameters
Umin, Umax = 0., 1.05
xlaboff = 0.7
ylab = 0.83
colors = plt.cm.coolwarm(np.linspace(0, 1, nM))

# Plot
plt.figure(figsize=(5, 2.5),dpi=100)
plt.axvline(kv_fid, color='black', ls='--')
plt.axvline(ks_fid, color='black', ls='--')
plt.text(xlaboff*kv_fid, 0.55, r'$1/r_\mathrm{v}$')
plt.text(xlaboff*ks_fid, 0.95, r'$1/r_\mathrm{s}$')
for iM in [0, (nM-1)//2, -1]:
    plt.plot(np.nan, color=colors[iM],  label='$10^{%d}\,h^{-1}M_\odot$'%(np.log10(Ms[iM])))
for i, _ in enumerate(Wks_M[0, :]): # TODO: Better
    plt.plot(ks, Wks_M[:, i], color=colors[i])
plt.xscale('log')
plt.xlabel(r'$k$ / $h\,\mathrm{Mpc}^{-1}$')
plt.xlim((kmin, kmax))
plt.ylabel(r'$\hat{U}_\mathrm{m}(M, k)$')
plt.ylim((Umin, Umax))
plt.legend()
plt.tight_layout()
plt.savefig('plots/NFW_U.pdf', bbox_inches='tight')
plt.show()

Now we compare $\hat U$ functions defined with fixed concentration or fixed virial radius with the previous plot

In [ ]:
# Fill Wk arrays with varying M
Wk_fid = halo.window_function(ks, rv_fid, c_fid, profile='NFW')
Wks_rv = halo.window_function(ks, rvs, c_fid, profile='NFW')
Wks_c = halo.window_function(ks, rv_fid, cs, profile='NFW')

In [ ]:
# Plot
plt.subplots(4, 1, sharex=True, figsize=(6.4, 8.4))

xlaboff = 0.6
ylab = 0.83

plt.subplot(4, 1, 1)
plt.axvline(kv_fid, color='black', ls='--')
plt.axvline(ks_fid, color='black', ls='--')
plt.text(xlaboff*kv_fid, ylab, r'$\pi/r_\mathrm{v}$')
plt.text(xlaboff*ks_fid, ylab, r'$\pi/r_\mathrm{s}$')
plt.plot(ks, Wk_fid, color=colors[(nM-1)//2])
plt.xscale('log')
plt.xticks([])
plt.xlim((kmin, kmax))
plt.ylabel(r'$U_\mathrm{m}(M, k)$')
plt.ylim((0., 1.1))

plt.subplot(4, 1, 2)
plt.text(0.5, 0.3, r'varying c and $r_{\rm v}$')
plt.plot(np.nan, color=colors[0],  label='$10^{%d}\,h^{-1}M_\odot$'%(np.log10(Ms[0])))
plt.plot(np.nan, color=colors[-1], label='$10^{%d}\,h^{-1}M_\odot$'%(np.log10(Ms[-1])))
for i, _ in enumerate(Wks_M[0, :]):
    plt.plot(ks, Wks_M[:, i], color=colors[i])
plt.xscale('log')
plt.xlim((kmin, kmax))
plt.xticks([])
plt.ylabel(r'$U_\mathrm{m}(M, k)$')
plt.ylim((0., 1.1))
plt.legend()

plt.subplot(4, 1, 3)
plt.text(0.5, 0.3, r'fixed c and varying $r_{\rm v}$')
for i, _ in enumerate(Wks_rv[0, :]):
    plt.plot(ks, Wks_rv[:, i], color=colors[i])
plt.xscale('log')
plt.xlim((kmin, kmax))
plt.xticks([])
plt.ylabel(r'$U_\mathrm{m}(M, k)$')
plt.ylim((0., 1.1))

plt.subplot(4, 1, 4)
plt.text(0.5, 0.3, r'varying c and fixed $r_{\rm v}$')
for i, _ in enumerate(Wks_c[0, :]):
    plt.plot(ks, Wks_c[:, i], color=colors[i])
plt.xscale('log')
plt.xlabel(r'$k$ / $h\,\mathrm{Mpc}^{-1}$')
plt.xlim((kmin, kmax))
plt.ylabel(r'$U_\mathrm{m}(M, k)$')
plt.ylim((0., 1.1))

plt.show()